In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [2]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 10

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

In [7]:
for batch_size in [64, 128, 256]:
    for optimizer in ["RMSprop", "Adam"]:
        model = Sequential()
        model.add(Conv2D(32, (3, 3), padding='same',
                         input_shape=x_train.shape[1:]))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))
        model.summary()
        print(f"batch size = {batch_size}, optimizer = {optimizer}")
        opt = Adam()
        if optimizer == "RMSprop":
            opt = RMSprop()
        model.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])

        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test))
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 64)       

Epoch 8/10
50000/50000 [==============================] - 103s 2ms/step - loss: 0.6877 - accuracy: 0.7605 - val_loss: 0.6792 - val_accuracy: 0.7667
Epoch 9/10
50000/50000 [==============================] - 101s 2ms/step - loss: 0.6619 - accuracy: 0.7683 - val_loss: 0.6858 - val_accuracy: 0.7658
Epoch 10/10
50000/50000 [==============================] - 101s 2ms/step - loss: 0.6259 - accuracy: 0.7818 - val_loss: 0.6911 - val_accuracy: 0.7676
Test loss: 0.6910939316749573
Test accuracy: 0.7675999999046326
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_19 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 30, 30, 32)        9248      
_______

50000/50000 [==============================] - 94s 2ms/step - loss: 0.9548 - accuracy: 0.6642 - val_loss: 0.8681 - val_accuracy: 0.6914
Epoch 5/10
50000/50000 [==============================] - 94s 2ms/step - loss: 0.8738 - accuracy: 0.6933 - val_loss: 0.7844 - val_accuracy: 0.7255
Epoch 6/10
50000/50000 [==============================] - 98s 2ms/step - loss: 0.8099 - accuracy: 0.7136 - val_loss: 0.7588 - val_accuracy: 0.7344
Epoch 7/10
50000/50000 [==============================] - 95s 2ms/step - loss: 0.7588 - accuracy: 0.7325 - val_loss: 0.7089 - val_accuracy: 0.7507
Epoch 8/10
50000/50000 [==============================] - 91s 2ms/step - loss: 0.7093 - accuracy: 0.7510 - val_loss: 0.7072 - val_accuracy: 0.7556
Epoch 9/10
50000/50000 [==============================] - 91s 2ms/step - loss: 0.6818 - accuracy: 0.7584 - val_loss: 0.7011 - val_accuracy: 0.7592
Epoch 10/10
50000/50000 [==============================] - 91s 2ms/step - loss: 0.6495 - accuracy: 0.7704 - val_loss: 0.6580 - va

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 90s 2ms/step - loss: 1.7239 - accuracy: 0.3628 - val_loss: 1.3784 - val_accuracy: 0.5008
Epoch 2/10
50000/50000 [==============================] - 90s 2ms/step - loss: 1.3322 - accuracy: 0.5209 - val_loss: 1.1550 - val_accuracy: 0.5893
Epoch 3/10
50000/50000 [==============================] - 91s 2ms/step - loss: 1.1520 - accuracy: 0.5905 - val_loss: 0.9898 - val_accuracy: 0.6464
Epoch 4/10
50000/50000 [==============================] - 88s 2ms/step - loss: 1.0297 - accuracy: 0.6354 - val_loss: 0.8966 - val_accuracy: 0.6840
Epoch 5/10
50000/50000 [==============================] - 86s 2ms/step - loss: 0.9482 - accuracy: 0.6667 - val_loss: 0.8400 - val_accuracy: 0.7048
Epoch 6/10
50000/50000 [==============================] - 88s 2ms/step - loss: 0.8690 - accuracy: 0.6944 - val_loss: 0.7806 - val_accuracy: 0.7295
Epoch 7/10
50000/50000 [==============================] - 94s 2ms/st

1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?

1. batch_size 在 128 時的表現最好. optimizer Adam 的表現比 RMS 好

2. DNN 只有三層參數就比 CNN 六層還多. 另外可以看到 CNN 的參數大多集中在 Dense 部分. 可見主要差異在 Conv 層需要的參數遠小於 Dense 層.